Image search with approximate nearest neighbors
====

[Annoy](https://github.com/spotify/annoy) is a library for approximate nearest neighbor search. In this kernel we are going to demonstrate the creation of a forest model from our images, and make some queries to see what comes out. It is a follow-up from a [previous experiment](https://www.kaggle.com/nicw102168/a-stroll-through-the-neighborhood-manifold) where we performed "brute-force" NN search.

It should be made clear that this is not a feature-based image search that we are going to try here. We are studying very small images, and doing direct comparison. So the features fed to the random forests are the image pixels themselves. This is not crazy because the datset has enough points to nicely sample the a neighborhood, as we demosntrated previously. But what will come out of a less restrictive neighborhood search?

## Some thoughts on nearest neighbor search
Nearest neighbor search is really one of the most important principles in machine learning and data analysis, and it's usually a good idea to see what it has to offer in any application you may be working on. Someitmes it is easy to get lost using different techniques, and you may even forget why you were not using a KNN classifier at a problem... So it's important to have some clarity about when you could or should use it, or not.

The first reason you might like to pick an alternative to KNN in a classificaiton problem is just about computational complexity: a different model may just consume less memory, or make faster inferences. Another reason might be that you can better enforce special constraints of your problem by using a different model. For instance, you may know for sure your problem has some physical constraints that you can easily enforce with a special model, or you have a great set of features that you know for sure will crate a good space where you can just use SVM. Or maybe your problem is noisy in a way that KNN doesn't let you achieve the best accuracy. Or the KNN cannot be obviously simplified in a way that retains a good accuracy, etc...

In many cases, though, KNN can give great results within your computational budget, and this is specially true even since the emergence of new techniques for approximate nearest neighbor search, such as utilized in Annoy. Today these libraries are being deployed in many places for classification, recommendation and other tasks, offering great performance coupled with great theoretical soundness.

Image search in random forests like this is usually done with some kind of feature extraction first. You are usually interested in some sort of detection of objects, so you need features that allow a distance comparison to tell you wether an image might contain a "signature" of an object. In our case here we are going to do something a bit different, we are really looking at the distance of the images in pixel space, really the "residue" from subtracting two images.

What we will do here is pretty much what we have done in our previous kernel, except now we are not looking at a very strict neighborhood calculating a maximum error pixel-by-pixel (Inifnity norm), but using a more conventional Eucliden distance, and looking for all the nearest neighbors, regardless of the distance. We are also using a better tool for the job!

## Training some forests and shaking the trees
So let's go for it. We start with some familiar definitions,

In [ ]:
import numpy as np
import numpy.linalg
import matplotlib.pyplot as plt
%matplotlib inline   
plt.rcParams['image.cmap'] = 'gray'

import annoy

In [ ]:
def image_generator(*filenames):
    for filename in filenames:
        with open(filename, 'rb') as fp:
            for _ in range(100000):
                yield np.array(np.fromstring(fp.read(401), dtype=np.uint8)[1:], dtype=float)

def img_gen(batches=10):
    return image_generator(*["../input/snake-eyes/snakeeyes_{:02d}.dat".format(nn) for nn in range(batches)])

def plotdice(x):
    plt.imshow(x.reshape(20,20))
    plt.axis('off')                
def plotdicetiling(xx):
    rows = len(xx) // 10
    plt.figure(figsize=(12, rows * 6 // 5))
    for n, q in enumerate(xx):
        plt.subplot(rows, 10, n+1)
        plotdice(q)
def plotdicez(x):
    plt.imshow(x.reshape(20,20), cmap=plt.cm.RdBu, vmin=-128, vmax=128)
    plt.axis('off')

We'll just pretty much copy the README example from the Annoy home page here, and see what happens. You train the model by "feeding" the data point by point, and then asking the object to build the forst. But we'll start with a small model first, just 100.000 images from our first batch, and a single tree.

In [ ]:
vector_length = 400
Ntrees = 1
t = annoy.AnnoyIndex(vector_length)
for i, v in zip(range(100000), img_gen(1)):
    t.add_item(i, v)
t.build(Ntrees)

We'll now query for the first 6 images in the dataset. 

In [ ]:
for ref in range(6):
    neighbors = t.get_nns_by_item(ref, 50)
    plotdicetiling([np.array(t.get_item_vector(i)) for i in neighbors])

The first thing to notice is how it managed to deliver us plenty of images where the dice are kind of sititng in the same geometric configuration, although with some confusion about the faces. I would say that is pretty much what I expected here, specially considering the norm is more relaxed than in our previous experiment. So we are allowing some incorrect spots in the face pattern if the dice is well aligned to the reference, for instance.

It is impossible not to notice how the faces are quite bad in the case of two dice, while for single dice a 1-NN classification might not give a terrible result.

Let's use more data now, and a more complex model.

In [ ]:
Ntrees = 10
tbig = annoy.AnnoyIndex(vector_length)
for i, v in zip(range(1000000), img_gen(10)):
    tbig.add_item(i, v)
tbig.build(Ntrees)

In [ ]:
for ref in range(6):
    neighbors = tbig.get_nns_by_item(ref, 50)
    plotdicetiling([np.array(tbig.get_item_vector(i)) for i in neighbors])

We got some improved results, pretty much as expected. Quite nice to see how it just nails the classification in the last two cases. The class is just '1', but we managed to find 49 other images with a '1' and pretty much the same geometric configuration. For the '3' and '2' we got a lot more of the correct class, and the pattern is even well-aligned.

For the pairs of dice, it really looks quite challenging to find really good matches. Apparently they are just not there in the dataset. We probably need to increase the sample size to something above 10M to really find very similar inputs. We can notice an improvement in the gemetric disposition of the dice here, though, and the retrieved face classes are a little less diverse. So it does look like it "narrowed it down" a bit.




## Messing it up
We have seen that we get better results using more data and a larger model, but how does it degrade if we constrain a lot the amount of data?

In [ ]:
Ntrees = 10
tbad = annoy.AnnoyIndex(vector_length)
for i, v in zip(range(1000), img_gen(1)):
    tbad.add_item(i, v)
tbad.build(Ntrees)
for ref in range(6):
    neighbors = tbad.get_nns_by_item(ref, 50)
    plotdicetiling([np.array(tbad.get_item_vector(i)) for i in neighbors])

It is a quite interesting result to see. With so little points, we get results that can bear very little resemblance with the input. The geometry is not sacred anymore, a dice may even be rotated at any angle in there. You get quite a variety of faces, and there is even a lot of confusion between one-dice and two-dice images! It would be interesting to look later at a quantitative evaluation of these outputs as a function of the volume of data and model complexity.

## Conclusion
We have demonstrated how a modern approximate nearest-neighbor library like Annoy is quite capable of handling a dataset like Snake-Eyes, with a million tiny images, allowing us to query the dataset and obtain a sample that does approximate the input as we might expect: similar geometry and texture. This result depends on the volume of data and model complexity, and naturally there is a trade-off with computation time ath the model building and querying.

In our next steps studying this dataset we'll build a classifier using this approximate-nearest neighbor. And later we're interested to see how its complexity compares to other models, specially convolutional neural networks, and then how such a random forest could be extended or specialized to fit this specific problem of image classification.